# Electromagnetic field evolution in 1D

This notebook introduces the fundamental principles of how the evolution of the electric and magnetic fields over time can be computed. As a first case, we will ignore electric charges (charged particles) and thus currents and stick to one-dimensional simulations. Later notebooks will extend on the algorithms presented here. 

### Remark:
All notebooks here use [`python`](https://www.python.org/) as a programming language. This is not the most performant choice since `python` is an interpreted language. To make matters even worse, we will not use efficient modules like [`numpy`](https://www.numpy.org/) in order to demonstrate the algorithms clearly and not obfuscate them by writing them as efficient `numpy` function calls. 

## Maxwell's equations

Classical electrodynamics, the interplay between electric and magnetic fields and how they are changed based on charges and currents, is described by the four [Maxwell's equations](https://en.wikipedia.org/wiki/Maxwell%27s_equations).

$
\begin{eqnarray}
\vec{\nabla} \vec{E} & = & \frac{\rho}{\epsilon_0} \tag{1}\\
\vec{\nabla} \vec{B} & = & 0 \tag{2} \\
\vec{\nabla} \times \vec{E} & = & - \frac{\partial \vec{B}}{\partial t} \tag{3} \\
\vec{\nabla} \times \vec{B} & = & \mu_0 \vec{J} - \mu_0 \epsilon_0 \frac{\partial \vec{E}}{\partial t} \tag{4}
\end{eqnarray}
$

with $\vec{E}$ being the electric and $\vec{B}$ being the magnetic field. $\vec{J}$ is the current and $\rho$ the charge density. The differential operator $\vec{\nabla}$ defines the divergence and rotation and can be written as 

$$
\vec{\nabla} = \begin{bmatrix}\vec{e}_x \frac{\partial}{\partial x} \\  \vec{e}_y \frac{\partial}{\partial y} \\ \vec{e}_z \frac{\partial}{\partial z} \end{bmatrix} 
$$

$\epsilon_0$ and $\mu_0$ are constants. The first is called [vacuum permittivity](https://en.wikipedia.org/wiki/Vacuum_permittivity) or electric constant. The second is named the [vacuum permeability](https://en.wikipedia.org/wiki/Vacuum_permeability) or magnetic constant. They are related to the speed of light $c$ by the following relationship:

$$
c^2 = \frac{1}{\epsilon_0 \mu_0}
$$


### Time evolution

Assuming we have a valid solution of Maxwell's equation for both the electric $\vec{E}(\vec{r},t)$ and magnetic field $\vec{B}(\vec{r},t)$ for the entire  volume of interest for time $t = 0$. 
Based on that description, we need to integrate the time evolution of the electric and magnetic fields to predict their evolution over time. 
The 3rd and 4th Maxwell's equations contain a time derivate of the fields.  If we rearrange both equation, we get the following system of coupled particle differential equations ([PDE](https://en.wikipedia.org/wiki/Partial_differential_equation)):

$
\begin{eqnarray}
\frac{\partial \vec{E}}{\partial t} & = & - \frac{1}{c^2} \vec{\nabla} \times \vec{B} + \frac{1}{\epsilon_0} \vec{J} \tag{5}\\
\frac{\partial \vec{B}}{\partial t} & = & - \vec{\nabla} \times \vec{E} \tag{6}\\
\end{eqnarray}
$

By integrating the time evolution, we can determine the field distribution at a later (earlier) time. 

$$
\begin{eqnarray}
\vec{E}(t) & = & \vec{E}(0) + \int \limits_0^{t} \left( - \frac{1}{c^2} \vec{\nabla} \times \vec{B}(\tau) + \frac{1}{\epsilon_0} \vec{J}(\tau) \right) \mathrm{d}\tau \\
\vec{B}(t) & = & \vec{B}(0) + \int \limits_0^{t} \left( - \vec{\nabla} \times \vec{E} \right) \mathrm{d}\tau \\
\end{eqnarray}
$$

### Component-wise formulation of the  Maxwell equation

In [1]:
import matplotlib.pyplot as plt